<a href="https://colab.research.google.com/github/Partha-deuri/learn_pytorch/blob/main/webScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
import requests
from bs4 import BeautifulSoup
import re

In [210]:
#initialize
globalCorpus = set()
globalLinks = set()
globalVisited = set()


In [211]:
# helpers
skipLines = [
        '© asomiyapratidin 2024',
        "অসমৰ সৰ্বাধিক প্ৰচলিত দৈনিক বাতৰিকাকত। ২০১১ চনৰ ৪ মাৰ্চত বাতৰিকাকতখন প্ৰথম প্ৰকাশ পায়। কাকতখনৰ সম্পাদক নৰেশ কলিতা। বাতৰি কাকতখনৰ চাৰিটা সংস্কৰণ গুৱাহাটী, ডিব্ৰুগড়, বিশ্বনাথ চাৰিআলি আৰু গোৱালপাৰাৰ পৰা একেসময়তে প্ৰকাশ পায়।"
    ]
def printList(list):
    for item in list:
        print(item)


In [212]:
# get all links

def fetchAllLinks (url,nested=10):
    if nested == 0:
        return
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    res=soup.find_all('a')
    for item in res:
        link = item.get('href')
        if link!=None and link!='#':
            if re.search("^"+rootPageUrl+".*", link):
                if re.search(".*(login|privacy|contact|terms-conditions).*",link):
                    continue
                if link not in globalLinks:
                    globalLinks.add(link)
                    fetchAllLinks(link,nested-1)


In [213]:
#get data from one page
def fetchData(link):
    corpus = []
    tempReq = requests.get(link)
    tempSoup = BeautifulSoup(tempReq.content,"html.parser")
    tempRes = tempSoup.findAll('p')
    for p in tempRes:
        para = p.text
        if para not in skipLines:
            para = re.sub(r"[^ঀ-ৱ]+",' ',para)
            if para !=' ':
                corpus.append(para)
    return corpus


In [214]:
# get data from all pages
def  getAllData(links=[]):
    for link in links:
        if link not in globalVisited:
            globalVisited.add(link)
            data = fetchData(link)
            if len(data) >0:
                for d in data:
                    globalCorpus.add(d)

In [215]:
#fix additional '\n'
def fixIndent(filename):
    f = open(filename+".txt","r",encoding='utf-8')
    newFile = f.readlines()
    f.close()
    # print(newFile)
    f = open(filename+".txt","w",encoding='utf-8')
    for line in newFile:
        if line != '\n':
            f.write(line)
    f.close()


In [216]:
# Create file
def createFile(filename):
    f = open(filename+".txt", "a", encoding='utf-8')
    for para in globalCorpus:
        # for para in doc:
        para = para.split('।')
        for line in para:
                # line = removeRedundent(line)
            line = re.sub(r"[^ঀ-ৱ]+",' ',line)
            line = re.sub('^ +','',line)
            f.write(line)
            f.write("\n")
    f.close()
    fixIndent(filename)


In [217]:

#initialize
globalCorpus = set()
globalLinks = set()
globalVisited = set()

rootPageUrl = 'https://www.asomiyapratidin.in/'
globalLinks.add(rootPageUrl)

fetchAllLinks(rootPageUrl)

getAllData(globalLinks)

# printList(globalCorpus)

createFile("assamese_corpus")

print("\n\n\t\t***** Task Completed *****\n\n")



		***** Task Completed *****




In [218]:
# f = open("assamese_corpus.txt","r",encoding='utf-8')
# print(f.read())
# f.close()

In [219]:
globalLinks = set()
globalVisited = set()
globalCorpus = set()

rootPageUrl = 'https://ganaadhikar.com/'
startPageUrl = 'https://www.ganaadhikar.com/'
globalLinks.add(rootPageUrl)

# printList(globalLinks)

fetchAllLinks(startPageUrl)
# printList(globalLinks)
getAllData(globalLinks)
# printList(globalCorpus)
createFile("assamese_corpus")

print("\n\n\t\t***** Task Completed *****\n\n")

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)




		***** Task Completed *****




In [220]:
globalCorpus = set()
globalLinks = set()
globalVisited = set()

rootPageUrl = 'https://niyomiyabarta.com/'
startPageUrl = 'https://www.niyomiyabarta.com/home/'
globalLinks.add(startPageUrl)

fetchAllLinks(startPageUrl)
# printList(globalLinks)
getAllData(globalLinks)
# printList(globalCorpus)

createFile("assamese_corpus")

print("\n\n\t\t***** Task Completed *****\n\n")



		***** Task Completed *****




In [221]:
# fetchData('https://ganaadhikar.com/%e0%a6%9c%e0%a7%8b-%e0%a6%ac%e0%a6%be%e0%a6%87%e0%a6%a1%e0%a7%87%e0%a6%a8-%e0%a6%ad%e0%a7%b1%e0%a6%bf%e0%a6%b7%e0%a7%8d%e0%a6%af%e0%a6%a6%e0%a7%8d%e0%a6%ac%e0%a6%be%e0%a6%a3%e0%a7%80/')

In [222]:
line = "dfbsjdhfbj ক্ষতিগ্ৰস্তৰ আশ্ৰয় শিবিৰত  dbfsdbমন্ত্ৰী পীয়ূষ হাজৰিকা dhfjh"
line = re.sub(r"[^ঀ-ৱ]+",' ',line)
print(line)

 ক্ষতিগ্ৰস্তৰ আশ্ৰয় শিবিৰত মন্ত্ৰী পীয়ূষ হাজৰিকা 
